# 高大連携プロジェクト
## 「カシマスタジアム行きバスの経由地最適化」

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd 
import pulp

In [2]:
# 定数
place = list(range(2, 15))   # 経由地候補
S = 1    #　スタート（場所は任意）
G = 15    # ゴール(カシマスタジアム)
bus = 3    # バスの台数
M = 300
# 入力データ
distance = pd.read_csv('distance.csv', header=None)    # 経由地候補間の距離行列
demand = pd.read_csv('demand.csv', header=None)    # 経由地候補の需要

# 変数
## x : ０-１変数
x = {(i, j):pulp.LpVariable('x({},{})'.format(i, j), cat='Binary') for i in range(1,15) for j in range(2,16)}
## t : 時間
t = {(i):pulp.LpVariable('t({})'.format(i), lowBound=0, cat='Integer') for i in range(2,15)}
## y : バスの乗車人数
y = {(i):pulp.LpVariable('y({})'.format(i), lowBound=0, cat='Integer') for i in range(1,15)}
## z : 目的変数
z = {(i):pulp.LpVariable('z({})'.format(i), lowBound=0, cat='Integer') for i in range(1,15)}

In [3]:
#x = [pulp.LpVariable('x' , 0, 1, 'Binary')] #
#t = [pulp.LpVariable.dicts('t' , 0, 1, 'Integer')] # 
#y = [pulp.LpVariable.dicts('y' , 0, 1, 'Integer')] # バスの乗車人数
#z =  [pulp.LpVariable.dicts('z' , 0, 1, 'Integer')] # 目的変数

In [4]:
# 問題宣言
prob = pulp.LpProblem("kashima_bus_routes", sense=pulp.LpMaximize)
# 目的関数
prob += pulp.lpSum(z[i] for i in place)

In [10]:
# 制約式
# 
prob += pulp.lpSum(x[S,i] for i in range(2,16)) == bus
prob += pulp.lpSum(x[i,G] for i in range(1,15)) == bus

for j in place:
    prob += pulp.lpSum(x[i,j] for i in range(1,15)) ==  pulp.lpSum(x[j, k] for k in range(2,16))
    prob += pulp.lpSum(x[i,j] for i in range(1,15)) <= 1

for i in place:
    prob += x[i,i] == 0
    prob += x[S,G] <= 0
    prob += x[S,G] >= 0
    
#  スタート時の乗客は0人
prob += y[S] == 0

In [21]:
for i in range(1,15)
    for j in range(2,15):
        prob += - M * (1-x[i, j]) + y[j] <= y[i] + demand[j]

KeyError: 13

In [12]:
#
for i in place:
    prob += -M*(1-x[i, G]) + z[i] <= y[i]

In [13]:
# 乗車人数は25人以上，45人以下
for i in range(1,15):　
    prob += z[i] <= 45*x[i,G] 
    prob += 25*x[i,G] <=  z[i]

SyntaxError: invalid character in identifier (<ipython-input-13-726db2b49fbb>, line 2)

In [18]:
# 
for i in place:
    for j in place:
        prob += -200*(1-x[i,j])+distance[i, j] <= t[i]

KeyError: (2, 2)

In [19]:
# 
for i in range(2,15):
    prob += -200*(1-x[i, G])+distance[i, G] <= t[i]

KeyError: (2, 15)

In [20]:
# 自家用車で直接行く場合の1.5倍以内に収める
for i in (2,15):
    prob += t[i] <= 1.5*distance[i, G]

KeyError: (2, 15)